## Localizing a face with a face detector
In Lab4, we built a face detector we still have a few steps in order to be able to locate face

In [1]:
import tensorflow as tf
import keras

print(tf.__version__)
# load our best trained model 
model = tf.keras.models.load_model('MLPmodel_Best94.h5')
# model.summary()

Using TensorFlow backend.


2.3.1


In [2]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import math
import cv2 as cv2
from skimage.transform import pyramid_gaussian

In [3]:
def exact_windows(image,size):
    x = 0
    y = 0
    image = []
    cordinates = []
    while x < (image.shape[1]-size[1]):
        while y < (image.shape[0]-size[0]):
            res = image[y:y+size[0],x:x+size[1]]
            res = cv2.resize(res,[24,24])
            image.append(res)
            cordinates.append([y,x,y+size[0],x+size[1]])
            y = y + (int)[size[0]*1/4]
        x = x + (int)[size[1]*1/4]
        y =0
    return (image,cordinates)

def detec_faces(image,model,windows_sizes):
    windows = []
    windows_coord = []
    for size in windows_sizes:
        res1,res2 = exact_windows(image,size)
        for img in res1:
            windows.append(img)
        windows_coord+=res2
    windows = np.array(windows)
    res = np.argmax(model.predict(windows),axis=1)
    return (res,windows_coord)

In [4]:
# compute the center of gravity
def centerOfGravaity(rectangles):
    center = [0,0]
    sigma = [0,0]
    np_points = 0
    for rectangle in rectangles:
        ty,tx,by,bx = rectangle
        for x in range(tx,bx+1):
            for y in range(ty,by+1):
                center[0] +=x
                center[1] +=y
                nb_points +=1
    center[0] = center[0]/nb_points
    center[1] = center[1]/nb_points

    #compute sigmaI and sigmaJ
    for rectangle in rectangles:
        ty,tx,by,bx = rectangle
        for x in range(tx,bx+1):
            for y in range(ty,by+1):
                sigma[0] += (x-center[0])**2
                sigma[1] += (x-center[1])**2
    sigma[0] = 2*(math.sqrt(sigma[0]/nb_points))+1
    sigma[1] = 2*(math.sqrt(sigma[1]/nb_points))+1

    #compute the top left and botton right points
    topx = int(center[0]-(sigma[0]/2))
    topy = int(center[1]-(sigma[1]/2))
    botx = int(center[0]+(sigma[0]/2))
    boty = int(center[1]+(sigma[1]/2))

    return (topx,topy,botx,boty)

In [5]:
def methodPyramid(img,model):
    window_sizes=[(16,16),(32,32)]
    true_rect = []
    i=0
    pyramid = tuple(pyramid_gaussian(img,downscale=2,sigma=1,multichannel=True))
    while pyramidp[i].shape[0]>window_sizes[0][0] and pyramid[i].shape[1]>window_sizes[0][1]:
        res,res2=detect_faces(pyramid[i]*255,model,window_sizes)
        for y in range(len(res)):
            if res[y]==1:
                temp = [x*(2**1) for x in res2[y]]
                true_rect.append(temp)

    #localize the face
    if(len(true_rect)!=0):
        topx,topy,botx,boty = centerOfGravity(true_rect)
    else:
        topx,topy,botx,boty = 0,0,0,0
    return (topx,topy,botx,boty)

In [6]:
# Turn on camera to capture real time human faces
# import time 
# color = (255,0,0)
# thickness =2
# cap = cv2.VideoCapture(0) # take camera as input, pass 0

# while(True):
#     ret,frame = cap.read()
#     (topx,topy,botx,boty) = methodPyramid(frame,model)
#     image = cv2.rectangle(frame,(topx,topy),(botx,boty))

#     cv2.imshow('frame',image)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# cap.release()
# cv2.destroyAllWindows()

In [7]:
import cv2

# Blue color in BGR 
color = [(255, 0, 0),(0, 255, 0),(0, 0, 255), (255,255,255)]
  
# Line thickness of 2 px 
thickness = 2

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('/Users/junyi/Desktop/00UGA-M2/CV-M2R-UGA/Projects/Seq01-1P-S0M1/Video/Seq01-1P-S0M1_CAM1.mp4')
# cap = cv2.VideoCapture()

f = open("/Users/junyi/Desktop/00UGA-M2/CV-M2R-UGA/Projects/Seq01-1P-S0M1/GroundTruth/face_bb.txt", "r")
line = f.readline().split(',') 
 
# Check if camera opened successfully
if (cap.isOpened()== False):
  print("Error opening video stream or file")
 
nb_frame = 0
# Read until video is completed
while (cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        while line and nb_frame == int(line[0]):
          print(nb_frame, line)
          cv2.rectangle(frame, (int(float(line[2])),int(float(line[3]))), (int(float(line[2]))+int(float(line[4])) ,int(float(line[3]))+int(float(line[5]))), color[int(line[1])-1], thickness)
          # (571.87, 71.999) (78.749, 80.624)
          line = f.readline()
          if line:
            line = line.split(',') 
        # Display the resulting frame
        cv2.imshow('Face_Capture_Frame', frame)
 
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        nb_frame +=1    
    # Break the loop
    else:
        break

# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

57 ['57', '1', '572.62', '77.249', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
58 ['58', '1', '571.87', '71.999', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
59 ['59', '1', '559.12', '76.124', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
60 ['60', '1', '551.99', '76.874', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
61 ['61', '1', '545.24', '77.999', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
62 ['62', '1', '537.74', '79.874', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
63 ['63', '1', '530.62', '82.874', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
64 ['64', '1', '523.12', '85.499', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
65 ['65', '1', '515.24', '88.499', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
66 ['66', '1', '507.37', '91.499', '78.749', '80.624', '-999', '-1', '-1', '-1\n']
67 ['67', '1', '499.87', '91.124', '64.124', '72.749', '-999', '-1', '-1', '-1\n']
68 ['68', '1', '494.62', '92.249', '64.124', '72.749', '-999', '-1', '-1', '-1\n']
69 [